# Preprocessing Events data
In this notebook, fist step on our preprocessing pipeline, we'll gather the data scraped from the several platforms we looked up, which obviously comes in a handful of shapes with different informations, s we'll have to slightly process it in order for all data to fit in the same structure.

In [1]:
import pandas as pd
import os
import glob
import urllib
import requests
import time
import json
from pandas.io.json import json_normalize
from IPython.display import clear_output
import numpy as np
import bandsInTownHelper as bandsInTownHelper

import pycountry
import country_demonyms

#### Loading data from source CSVs
Recall the four sources from which we acquired our data, with their benefits and flaws :
[Events.ch](www.events.ch), a non-specific platform for Swiss entertainment events
[BandsInTown](www.bandsintown.com), an American platform for popular music events 
[ResidentAdvisor](www.residentadvisor.net), an British platform for electronic music
[RouteDesFestivals](www.routedesfestivals.com), a French platform for music festivals happening in Europe
Here, we will not adress the specific issues of each source, but rather shape the data to fit in a single set.

In [7]:
#Load the total_x events data into several DataFrame.

total_eventsch = pd.read_csv(os.path.join('./total_eventsch.csv'))
total_bands_in_town = pd.read_csv(os.path.join('./total_bands_in_town.csv'))
total_residentadvisor = pd.read_csv(os.path.join('./total_residentadvisor.csv'))
total_routedesfestivals = pd.read_csv(os.path.join('./total_routedesfestivals.csv'))

### Preprocessing Events.ch data
One of the specific problem with face here, is that Artists are aggregated by events. That means that we get several artists per row (event), which is not conform to the other sources and raises several issues : what if several bands of different origins (probable) and genres (less probable) play together during the same event ? To solve this, we will expand each event to the number of artists represented in the Artist column. Classical music apart, it will be straightforward parsing around commas.

In [8]:
### Convert the date of events to Datetime type.
total_eventsch['Date'] = pd.to_datetime(total_eventsch['Date'])
total_eventsch['Date'] = total_eventsch['Date'].apply( lambda x: x.date() )

# We put classical music events aside : the artist information is problematic as it encompasses orchestras, conductors,
# solists and composers. Our analysis won't be finegrained with regard to classical music, unfortunately.
total_eventsch_classic = total_eventsch.loc[total_eventsch.Genre == 'Classic']
total_eventsch = total_eventsch.select(lambda x: x not in total_eventsch_classic.index)

# Create a new DF to store events with parsed Artists
df_columns=['Artist', 'Date', 'Genre', 'Venue', 'City']
total_eventsch_processed=pd.DataFrame(columns=df_columns)

for index, artist in zip(total_eventsch.index, total_eventsch.Artist) :
    for name in artist.split(',') :
        name=name.strip()
        if  (len(name) != 0 and name[0].isupper()) :
            new_row = pd.DataFrame([[name, total_eventsch.loc[index].Date, total_eventsch.loc[index].Genre, total_eventsch.loc[index].Venue, total_eventsch.loc[index].City]], columns = df_columns)
            total_eventsch_processed = total_eventsch_processed.append(new_row)
            
total_eventsch_processed.reset_index(drop = True, inplace=True)

Now, we will have to do a little of preprocessing to clean the Artist columns from unwanted characters, which would prevent the correct recognition of their names when we call on music databases API. As origin information may be included in the name cells (eg. (USA)), we will parse the rows for origin information before erasing the markers from names. 

In [9]:
# Find origin based on artist name
# Create a dict of Country adjective to Country name
country_dict = {}
for key, value in country_demonyms.COUNTRY_DEMONYMS.items():
    country_dict[value.lower()] = key.lower().title()

# Use PyCountry to get lists of country names and codes
country_name = []
country_alpha2 = []
country_alpha3 = []
for country in list(pycountry.countries) :
    if ' ' not in country.name :
        country_name.append(country.name)
    country_alpha2.append(country.alpha_2)
    country_alpha3.append(country.alpha_3)
country_alpha2.remove('DJ')
country_alpha2.remove('MC') 
country_dict['schweizer'] = 'Switzerland'

#Iterate the names and look for country information
for index, genre in zip(total_eventsch_processed.index, total_eventsch_processed.Artist) :
    for word in genre.split() :
        if word in country_name :
            total_eventsch_processed.set_value(index, 'origin', word)
        elif (word[1:-1] in country_alpha2 and '(' in word and ')' in word) :
            total_eventsch_processed.set_value(index, 'origin', pycountry.countries.lookup(word[1:-1]).name)
        elif (word[1:-1] in country_alpha3 and '(' in word and ')' in word) :
            total_eventsch_processed.set_value(index, 'origin', pycountry.countries.lookup(word[1:-1]).name)
        elif word.lower() in country_dict :
            total_eventsch_processed.set_value(index, 'origin', country_dict[word.lower()])

# Clean names by removing country / instrument information betwen parenthesis.
total_eventsch_processed.Artist.replace(' \(.*\) .*','', regex=True, inplace=True)
total_eventsch_processed.Artist.replace(' \(.*\)','', regex=True, inplace=True)
total_eventsch_processed.Artist.replace(' \(.*','', regex=True, inplace=True)

We concatenate the events back with the classical music ones into a single DF, and add venue-specific columns.

In [10]:
total_eventsch_parsed = pd.concat([total_eventsch_processed, total_eventsch_classic])
total_eventsch_parsed.rename(columns={'Genre': 'genre'}, inplace=True)
total_eventsch_parsed['Latitude'] = np.nan
total_eventsch_parsed['Longitude'] = np.nan
total_eventsch_parsed['Adress'] = np.nan
total_eventsch_parsed.head()

,Artist,City,Date,genre,Venue,origin,Latitude,Longitude,Adress
0,DJs Patric Pleasure,Basel,2017-01-20,"Hip Hop, R'n'B",Balz,NaN,NaN,NaN,NaN
1,Ramon Ramones,Basel,2017-01-20,"Hip Hop, R'n'B",Balz,NaN,NaN,NaN,NaN
2,Bülent Ceylan,Zürich,2017-01-19,"Ragga, Reggae, African Music, Dancehall",Vior Club,Germany,NaN,NaN,NaN
3,Raffi Lusso,Zürich,2017-01-19,"Ragga, Reggae, African Music, Dancehall",Vior Club,NaN,NaN,NaN,NaN
4,Miguel M,Zürich,2017-01-19,"Ragga, Reggae, African Music, Dancehall",Vior Club,NaN,NaN,NaN,NaN


Then, we export the artists of Events.ch. As we have a genre for each for them already, we don't need to submit them to music intelligence service such as Spotify, where only genre can be returned. Therefore, we'll process them apart from artists of other sources.

In [11]:
total_eventsch_artists = total_eventsch_processed.copy()
total_eventsch_artists.drop('Date', axis=1, inplace = True)
total_eventsch_artists.drop('Venue', axis=1, inplace = True)
total_eventsch_artists.drop('City', axis=1, inplace = True)
total_eventsch_artists.columns = ['name', 'genre', 'origin']
total_eventsch_artists["ambigous_result"] = np.nan
total_eventsch_artists["no_result"] = np.nan
total_eventsch_artists.drop_duplicates('name', inplace= True)
total_eventsch_artists.name.size

19179

In [ ]:
# Save parsed events and artists to CSVs

filename = 'total_eventsch_parsed.csv'
pd.DataFrame(total_eventsch_parsed, columns=list(total_eventsch_parsed.columns)).to_csv(filename, index=True, encoding="utf-8")
print('Total parsed event data from Events.ch saved to file')

filename = 'total_eventsch_artists.csv'
pd.DataFrame(total_eventsch_parsed, columns=list(total_eventsch_parsed.columns)).to_csv(filename, index=True, encoding="utf-8")
print('Total parsed event data from Events.ch saved to file')

### Preprocessing Bandsintown data
Very standard preprocessing, we remove extra informations grabbed from the API, normalize columns names and dates.

In [12]:
# Drop unnecessary informations
total_bands_in_town.drop(['artist_url', 'event_id', 'event_url', 'event_venue.region', 'event_venue.url', 'event_venue.id'],
                         1, inplace=True)

#Co nvert time column to datetime objects
total_bands_in_town['event_datetime'] = pd.to_datetime(total_bands_in_town['event_datetime'])
#Remove hours from dates
total_bands_in_town['event_datetime'] = total_bands_in_town['event_datetime'].apply( lambda x: x.date() )

# Renames columns and add missing ones
total_bands_in_town.columns = ['Artist', 'Date', 'City', 'Latitude', 'Longitude', 'Venue']
total_bands_in_town['genre'] = np.nan
total_bands_in_town['origin'] = np.nan
total_bands_in_town['Adress'] = np.nan
total_bands_in_town


total_bands_in_town.head(10)

,Artist,Date,City,Latitude,Longitude,Venue,genre,origin,Adress
0,Groombridge,2006-01-06,Langenthal,47.212060,7.789998,Rock in Church,NaN,NaN,NaN
1,Painhead,2006-01-07,Rorschach,47.477928,9.495190,Hafenbuffet,NaN,NaN,NaN
2,shEver,2006-01-14,Zug,47.182220,8.520760,Industrie 45,NaN,NaN,NaN
3,Painhead,2006-01-15,Gossau (Sankt Gallen),47.414415,9.254950,The Office,NaN,NaN,NaN
4,Mando Diao,2006-01-21,Laax,46.800000,9.250000,PALACE CLUB AT RIDERS PALACE,NaN,NaN,NaN
5,Foo Fighters,2006-01-25,Winterthur,47.495655,8.748480,Eishalle Duetwag,NaN,NaN,NaN
6,Groombridge,2006-01-27,Burgdorf,47.050000,7.616667,Gymfest,NaN,NaN,NaN
7,Groombridge,2007-01-01,Berne,46.948432,7.440461,ONO,NaN,NaN,NaN
8,Painhead,2007-01-06,Sommeri,47.566667,9.283333,Löwenarena,NaN,NaN,NaN
9,shEver,2007-01-13,Zurich,47.386620,8.534380,Werk21,NaN,NaN,NaN


### Preprocessing ResidentAdvisor data
ResidentAdvisor data comes with the full venue's adress, out of which we'll extract the city for coherence, and keep the adress for later viz.

In [14]:
# Extract cities from addresses of clubs in ResidentAdvisor data
for i, address in zip(total_residentadvisor.index, total_residentadvisor.club_adress) :
    total_residentadvisor.set_value(i, 'City', address.rsplit(None, 1)[-1])
    
# Manually fill wrong entries
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == '2' ].index.tolist(), 'City', 'Zurich')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == '1009' ].index.tolist(), 'City', 'Pully')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == '4001' ].index.tolist(), 'City', 'Basel')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == '1227' ].index.tolist(), 'City', 'Geneva')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == 'BL' ].index.tolist(), 'City', 'Münchenstein')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == '40/42' ].index.tolist(), 'City', 'Wetzikon')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == '22' ].index.tolist(), 'City', 'Bern')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == '/' ].index.tolist(), 'City', 'Murten')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == '253' ].index.tolist(), 'City', 'Les Diablerets')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == '14' ].index.tolist(), 'City', 'Basel')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == '10' ].index.tolist(), 'City', 'Basel')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == '-' ].index.tolist(), 'City', 'Biel/Bienne')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == '(GR)' ].index.tolist(), 'City', 'Klosters')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == 'ZH' ].index.tolist(), 'City', 'Zurich')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == '4058' ].index.tolist(), 'City', 'Basel')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == '1' ].index.tolist(), 'City', 'Baden')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == '6020' ].index.tolist(), 'City', 'Emmenbrücke')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == '8143' ].index.tolist(), 'City', 'Zurich')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == '2,6612,Ascona,(Ti),CH' ].index.tolist(), 'City', 'Ascona')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['City'] == 'ZG' ].index.tolist(), 'City', 'Baar ')

total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Rue des Grands-Vergers, 1957 Ardon, CH' ].index.tolist(), 'City', 'Ardon')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Av. de Tivoli 3, Fribourg, 1700, CH' ].index.tolist(), 'City', 'Fribourg')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Köniz, 3098, 9 Schulhausgässli, CH' ].index.tolist(), 'City', 'Köniz')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Unterer Graben 17, 9000 St. Gallen, CH' ].index.tolist(), 'City', 'St. Gallen')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Viale Castagnola 6, 6900 Lugano, CH' ].index.tolist(), 'City', 'Lugano')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Via Alla Foce 1, 6982 Agno, Ticino, CH' ].index.tolist(), 'City', 'Agno')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Via Pioda 12, 6900 Lugano, CH' ].index.tolist(), 'City', 'Lugano')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Ancienne-Pointe 16, 1920 Martigny, CH' ].index.tolist(), 'City', 'Martigny')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Freilager-Platz 9, 4142 Münchenstein/Basel, CH' ].index.tolist(), 'City', 'Basel')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Via Industria 4; 6814, Lamone Ticino' ].index.tolist(), 'City', 'Lamone')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Ponte Capriasca, Ticino' ].index.tolist(), 'City', 'Ponte Capriasca')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Case postale 352 Crans, Valais' ].index.tolist(), 'City', 'Crans-Montana')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Place centrale, 1997 Nendaz, Valais' ].index.tolist(), 'City', 'Nendaz')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Place de la Gare, 1957 Ardon, Valais, Suisse' ].index.tolist(), 'City', 'Ardon')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Rue du Commerce 122, 2300 La Chaux-de-Fonds, Suisse' ].index.tolist(), 'City', 'La Chaux-de-Fonds')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Place Centrale, 1870 Monthey, Valais, Suisse' ].index.tolist(), 'City', 'Monthey')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Chemin des Batailles, 1214 VERNIER (GENEVE), Suisse' ].index.tolist(), 'City', 'Vernier')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Rue de Vevey 34, 1630 Bulle, SWITZERLAND' ].index.tolist(), 'City', 'Bulle')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Sant Gallen, SWITZERLAND' ].index.tolist(), 'City', 'St. Gallen')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'Steinberggasse 16' ].index.tolist(), 'City', 'Winterthur')
total_residentadvisor.set_value(total_residentadvisor.loc[total_residentadvisor['club_adress'] == 'St. Annagasse 16' ].index.tolist(), 'City', 'Zurich')

# Drop events not in Switzerland
total_residentadvisor.drop(total_residentadvisor.loc[total_residentadvisor['City'] == 'Liechtenstein' ].index.tolist(), inplace=True)

# Convert events date to datetime objects
for i, date in zip(total_residentadvisor.index, total_residentadvisor.date) :
    total_residentadvisor.set_value(i, 'date', pd.to_datetime(date[5:]))
# Remove hour from dates
total_residentadvisor['date'] = total_residentadvisor['date'].apply( lambda x: x.date() )        

# Normalize columns names and add missing ones
total_residentadvisor.columns = ['Adress', 'Venue', 'Date', 'Artist', 'City']
total_residentadvisor['genre'] = np.nan
total_residentadvisor['origin'] = np.nan
total_residentadvisor['Latitude'] = np.nan
total_residentadvisor['Longitude'] = np.nan

total_residentadvisor.head(10)

,Adress,Venue,Date,Artist,City,genre,origin,Latitude,Longitude
0,Selnaustrasse 2,2. Akt Restaurant & Bar,2015-09-19,Affani,Zurich,NaN,NaN,NaN,NaN
1,Selnaustrasse 2,2. Akt Restaurant & Bar,2015-09-19,Mark Faermont,Zurich,NaN,NaN,NaN,NaN
2,Selnaustrasse 2,2. Akt Restaurant & Bar,2015-09-12,Mucho Stylez,Zurich,NaN,NaN,NaN,NaN
3,Selnaustrasse 2,2. Akt Restaurant & Bar,2015-07-03,Mucho Stylez,Zurich,NaN,NaN,NaN,NaN
4,Selnaustrasse 2,2. Akt Restaurant & Bar,2015-06-13,Mucho Stylez,Zurich,NaN,NaN,NaN,NaN
5,Selnaustrasse 2,2. Akt Restaurant & Bar,2015-03-21,Mark Faermont,Zurich,NaN,NaN,NaN,NaN
6,Selnaustrasse 2,2. Akt Restaurant & Bar,2015-01-17,Carlos Russo,Zurich,NaN,NaN,NaN,NaN
7,Selnaustrasse 2,2. Akt Restaurant & Bar,2015-01-17,Mark Faermont,Zurich,NaN,NaN,NaN,NaN
8,Selnaustrasse 2,2. Akt Restaurant & Bar,2014-11-29,Tonka,Zurich,NaN,NaN,NaN,NaN
9,Selnaustrasse 2,2. Akt Restaurant & Bar,2014-11-29,Mark Faermont,Zurich,NaN,NaN,NaN,NaN


### Preprocessing Route des Festivals data
For this source, we have to parse the date information, which is divided in three separate columns.

In [15]:
total_routedesfestivals = pd.read_csv(os.path.join('./total_routedesfestivals.csv'))
#make the three time columns into a single date column

total_routedesfestivals.month.unique()
for i, month in zip(total_routedesfestivals.index, total_routedesfestivals.month) :
    if month == 'Jan.':
        total_routedesfestivals.set_value(i, 'month', 1.0)
    if month == 'Fev.':
        total_routedesfestivals.set_value(i, 'month', 2.0)
    if month == 'Mar.':
        total_routedesfestivals.set_value(i, 'month', 3.0)
    if month == 'Avr.':
        total_routedesfestivals.set_value(i, 'month', 4.0)    
    if month == 'Mai':
        total_routedesfestivals.set_value(i, 'month', 5.0)   
    if month == 'Juin':
        total_routedesfestivals.set_value(i, 'month', 6.0)      
    if month == 'Juil.':
        total_routedesfestivals.set_value(i, 'month', 7.0)
    if month == 'Aout':
        total_routedesfestivals.set_value(i, 'month', 8.0)    
    if month == 'Sep.':
        total_routedesfestivals.set_value(i, 'month', 9.0)
    if month == 'Oct.':
        total_routedesfestivals.set_value(i, 'month', 10.0)        
    if month == 'Nov.':
        total_routedesfestivals.set_value(i, 'month', 11.0)   
    if month == 'Dec.':
        total_routedesfestivals.set_value(i, 'month', 12.0)
        
total_routedesfestivals.dropna(0, inplace=True)    

# Create datetime objects
total_routedesfestivals.month.apply(lambda x: pd.to_numeric(x))
total_routedesfestivals['Date'] = pd.to_datetime(total_routedesfestivals.year*10000 + total_routedesfestivals.month*100 + total_routedesfestivals.day, format="%Y%m%d")
total_routedesfestivals.drop(['day', 'month', 'year'], 1, inplace=True)

# We add this line to avoid type error on the datetimeindex when concatenating all frames
total_routedesfestivals['Date'] = pd.to_datetime(total_routedesfestivals['Date'])
# Remove hour from dates
total_routedesfestivals['Date'] = total_routedesfestivals['Date'].apply( lambda x: x.date() )

# Normalize columns names and add missing ones
total_routedesfestivals.columns = ['Venue', 'Artist', 'City', 'Date']
total_routedesfestivals['genre'] = np.nan
total_routedesfestivals['origin'] = np.nan
total_routedesfestivals['Latitude'] = np.nan
total_routedesfestivals['Longitude'] = np.nan
total_routedesfestivals['Adress'] = np.nan

total_routedesfestivals.head(10)

,Venue,Artist,City,Date,genre,origin,Latitude,Longitude,Adress
0,6 HOURS OF SYMPHONIA,SYNMETALIUM,Lausanne,2017-04-01,NaN,NaN,NaN,NaN,NaN
1,6 HOURS OF SYMPHONIA,EVENMORE,Lausanne,2017-04-01,NaN,NaN,NaN,NaN,NaN
2,6 HOURS OF SYMPHONIA,SECHEM,Lausanne,2017-04-01,NaN,NaN,NaN,NaN,NaN
3,6 HOURS OF SYMPHONIA,BEYOND FORGIVENESS,Lausanne,2017-04-01,NaN,NaN,NaN,NaN,NaN
4,ANTIGEL,ZERO,Geneve,2017-01-27,NaN,NaN,NaN,NaN,NaN
5,ANTIGEL,MAY B (MAGUY MARIN),Geneve,2017-01-28,NaN,NaN,NaN,NaN,NaN
6,ANTIGEL,MAY B (MAGUY MARIN),Geneve,2017-01-29,NaN,NaN,NaN,NaN,NaN
7,ANTIGEL,TRENTEMOLLER,Geneve,2017-02-02,NaN,NaN,NaN,NaN,NaN
8,ANTIGEL,THE NOTWIST,Geneve,2017-02-03,NaN,NaN,NaN,NaN,NaN
9,ANTIGEL,HENRI DES,Geneve,2017-02-05,NaN,NaN,NaN,NaN,NaN


## Aggregating events data
Finally, now that all our data is shaped homgeneously, we may concatenate everything into a convenient DataFrame and export it to be used farther along our pipeline. We have a total of 204815 events in our dataset.

In [16]:
total_events = pd.DataFrame(columns=total_eventsch_parsed.columns)
total_events = pd.concat([total_eventsch_parsed, total_bands_in_town, total_routedesfestivals, total_residentadvisor])

total_events.reset_index(drop = True, inplace=True)
total_events.index.size

204815

In [ ]:
#Write the DataFrame to a csv file
filename = 'total_events.csv'
pd.DataFrame(total_events, columns=list(total_events.columns)).to_csv(filename, index=False, encoding="utf-8")
print('Total events data saved to file')